<a href="https://colab.research.google.com/github/ttnok/c-programming-on-colab/blob/main/%E6%B5%AE%E5%8B%95%E5%B0%8F%E6%95%B0%E7%82%B9%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IEEE 754 binary64 ビット列観察

## 16 進表示

In [1]:
%%file fp.c
#include <stdio.h>

int main(void)
{
    unsigned char b[8];
    double *x = (double *)b;
 
    printf("? ");
    scanf("%lf", x);
 
    printf("b = %p\n", (void *)b);
    printf("x = %p\n", (void *)x);
    printf("*x = %f\n", *x);

    printf("%02x %02x %02x %02x %02x %02x %02x %02x\n",
           b[7], b[6], b[5], b[4], b[3], b[2], b[1], b[0]);

    return 0;
}



Writing fp.c


In [2]:
!clang fp.c -o fp

In [3]:
!./fp

? 1
b = 0x7ffc43a6558c
x = 0x7ffc43a6558c
*x = 1.000000
3f f0 00 00 00 00 00 00


## 2 進表示

In [4]:
%%file fp2.c
#include <stdio.h>
#include <stdlib.h>

int main(int c, char **v)
{
    unsigned char b[8];
    double *x = (double *)b;
 
    *x = strtod(v[1], NULL);

    for (int k = 0; k < 64; k++) {
        unsigned char mask = 0x80 >> (k % 8);
        printf("%d", (b[8 - k / 8 - 1] & mask) != 0);

        if (k % 4 == 3) printf(" ");
        if (k ==  0) printf(" / ");
        if (k == 11) printf("/ ");
    }

    return 0;
}



Writing fp2.c


In [5]:
!clang fp2.c -o fp2

In [6]:
!./fp2 1

0 / 011 1111 1111 / 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

In [7]:
!./fp2 -1

1 / 011 1111 1111 / 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

In [8]:
!./fp2 1.5

0 / 011 1111 1111 / 1000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

In [9]:
!./fp2 1.75

0 / 011 1111 1111 / 1100 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

## +0 と -0

In [10]:
!./fp2 0

0 / 000 0000 0000 / 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

In [11]:
!./fp2 -0

1 / 000 0000 0000 / 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

In [12]:
!./fp2 2

0 / 100 0000 0000 / 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

In [13]:
!./fp2 3

0 / 100 0000 0000 / 1000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

In [14]:
!./fp2 7

0 / 100 0000 0001 / 1100 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

## 0.1 + 0.2 と 0.3 の比較

In [15]:
!./fp2 0.1

0 / 011 1111 1011 / 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1010 

In [16]:
!./fp2 0.2

0 / 011 1111 1100 / 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1010 

In [17]:
!./fp2 0.3

0 / 011 1111 1101 / 0011 0011 0011 0011 0011 0011 0011 0011 0011 0011 0011 0011 0011 

## 丸め方式確認

In [29]:
!./fp2 1.1

0 / 011 1111 1111 / 0001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 1010 

In [30]:
!./fp2 2.1

0 / 100 0000 0000 / 0000 1100 1100 1100 1100 1100 1100 1100 1100 1100 1100 1100 1101 

In [31]:
!./fp2 4.1

0 / 100 0000 0001 / 0000 0110 0110 0110 0110 0110 0110 0110 0110 0110 0110 0110 0110 

In [32]:
!./fp2 8.1

0 / 100 0000 0010 / 0000 0011 0011 0011 0011 0011 0011 0011 0011 0011 0011 0011 0011 

In [39]:
%%file fenv.c
#include <stdio.h>
#include <math.h>
#include <fenv.h>

int main(void)
{
    int mode = fegetround();

    switch (mode) {
    case FE_DOWNWARD:
        puts("FE_DOWNWARD");
        break;
    case FE_UPWARD:
        puts("FE_UPWARD");
        break;
    case FE_TONEAREST:
        puts("FE_TONEAREST");
        break;
    case FE_TOWARDZERO:
        puts("FE_TOWARDZERO");
        break;
    default:
        printf("%d\n", mode);
        break;
    }

    return 0;
}

Overwriting fenv.c


In [40]:
!clang fenv.c -lm && ./a.out

FE_TONEAREST


In [18]:
!./fp2 9007199254740990

0 / 100 0011 0011 / 1111 1111 1111 1111 1111 1111 1111 1111 1111 1111 1111 1111 1110 

In [19]:
!./fp2 9007199254740991

0 / 100 0011 0011 / 1111 1111 1111 1111 1111 1111 1111 1111 1111 1111 1111 1111 1111 

In [20]:
!./fp2 9007199254740992

0 / 100 0011 0100 / 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

In [21]:
!./fp2 9007199254740993

0 / 100 0011 0100 / 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

## INF

In [22]:
%%file fp3.c
#include <stdio.h>
#include <stdlib.h>

int main(int c, char **v)
{
    unsigned char b[8];
    double *x = (double *)b;
 
    *x = 1.0 / 0.0;
    printf("%f\n", *x);

    for (int k = 0; k < 64; k++) {
        unsigned char mask = 0x80 >> (k % 8);
        printf("%d", (b[8 - k / 8 - 1] & mask) != 0);

        if (k % 4 == 3) printf(" ");
        if (k ==  0) printf(" / ");
        if (k == 11) printf("/ ");
    }

    return 0;
}



Writing fp3.c


In [23]:
!clang fp3.c && ./a.out

inf
0 / 111 1111 1111 / 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

## NaN

In [24]:
%%file fp4.c
#include <stdio.h>
#include <stdlib.h>

int main(int c, char **v)
{
    unsigned char b[8];
    double *x = (double *)b;
 
    *x = 0.0 / 0.0;

    printf("%f\n", *x);

    for (int k = 0; k < 64; k++) {
        unsigned char mask = 0x80 >> (k % 8);
        printf("%d", (b[8 - k / 8 - 1] & mask) != 0);

        if (k % 4 == 3) printf(" ");
        if (k ==  0) printf(" / ");
        if (k == 11) printf("/ ");
    }

    return 0;
}



Writing fp4.c


In [25]:
!clang fp4.c && ./a.out

nan
0 / 111 1111 1111 / 1000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 

## 非正規化数

In [26]:
%%file fp5.c
#include <stdio.h>
#include <stdlib.h>

int main(int c, char **v)
{
    unsigned char b[8];
    double *x = (double *)b;
 
    *x = 1.0e-309;

    printf("%.20f\n", *x);

    for (int k = 0; k < 64; k++) {
        unsigned char mask = 0x80 >> (k % 8);
        printf("%d", (b[8 - k / 8 - 1] & mask) != 0);

        if (k % 4 == 3) printf(" ");
        if (k ==  0) printf(" / ");
        if (k == 11) printf("/ ");
    }

    return 0;
}



Writing fp5.c


In [27]:
!clang fp5.c && ./a.out

0.00000000000000000000
0 / 000 0000 0000 / 0000 1011 1000 0001 0101 0111 0010 0110 1000 1111 1101 1010 1111 